In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
tshark_directory = os.path.join('.', 'traffictracer')
statistic_directory = ['WLAN_statistics', 'Meta_statistics'] 
ori_directory = ['WLAN', 'Meta'] 
port_directory = 'tshark_port' 
conn_directory = 'conn_in_out' 
evaluation_directory = 'evaluation'
data_date = '24-11-07'

meta_prefix = 'Meta-' 
wlan_prefix = 'WLAN-' 
conn_prefix = 'conn-in-out-'

In [3]:
direcrory_meta_statistic = os.path.join('.', 'wireshark_traffic', 'tshark', 'Meta_statistics') 
direcrory_meta = os.path.join('.', 'wireshark_traffic', 'tshark', 'Meta') 
direcrory_conn_in_out = os.path.join('.', 'wireshark_traffic', 'tshark', 'conn_in_out') 

conn_in_out_path = os.path.join(direcrory_conn_in_out, 'conn-in-out-24-09-26--17-03-48.csv') 
meta_statistic_path = os.path.join(direcrory_meta_statistic, 'Meta-24-09-26--16-59-01.csv') 
meta_path = os.path.join(direcrory_meta, 'Meta-24-09-26--16-59-01.csv') 

In [7]:
sni_dict = {
    'Name': [], 
    'AllSNI': [], 
    'FilteredSNI': []
}

In [8]:
for conn_info in os.listdir(os.path.join(tshark_directory, conn_directory)): 
    if conn_info.startswith(conn_prefix + data_date): 
        df_conn = pd.read_csv(os.path.join(tshark_directory, conn_directory, conn_info)) 
        sni_unique = set(df_conn['Server Name'].dropna().unique()) 
        sni_dict['Name'].append(conn_info[12:-4]) 
        sni_dict['AllSNI'].append(sni_unique) 
        sni_dict['FilteredSNI'].append('')

In [9]:
sni_csv = pd.DataFrame(sni_dict)

In [10]:
sni_csv.to_csv(os.path.join(tshark_directory, evaluation_directory, 'tt_stream_distribution', 'sni', data_date + '.csv'))

In [ ]:
for meta_file in os.listdir(os.path.join(tshark_directory, direcrory_meta)): 
    if meta_file.startswith('Meta' + data_date): 
        packets = pd.read_csv(os.path.join(tshark_directory, direcrory_meta, 'Meta' + data_date + '.csv')) # 防止读入同名pcap 
        packets = packets.dropna(subset=['TCP Stream index']) 
        packets_statistic = pd.read_csv(os.path.join(tshark_directory, direcrory_meta_statistic, 'Meta' + data_date + '.csv')) 
        packets_conn = pd.read_csv(os.path.join(tshark_directory, direcrory_conn_in_out, 'conn-in-out-' + data_date + '.csv')) 
        packets_server_name_unique = packets['Server Name'].dropna().unique() 
        domain_stream_all = {key: [] for key in packets_server_name_unique} 
        for name in domain_stream_all.keys(): # get the dictionary of sni: stream_ids
            stream_ids = packets.loc[packets['Server Name'] == name, 'TCP Stream index'] 
            domain_stream_all[name].extend(stream_ids) 
        stream_domain_all = {stream_id: domain for domain, stream_ids in domain_stream_all.items() for stream_id in stream_ids} # reverse the dictionary
        packets['Server Name'] = packets['TCP Stream index'].map(stream_domain_all) 
        packets_statistic['Server Name'] = packets_statistic['Stream ID'].map(stream_domain_all) 
        

In [11]:
packets = pd.read_csv(meta_path) # with tcp segment column
packets = packets.dropna(subset=['TCP Stream index']) 

packets_statistic = pd.read_csv(meta_statistic_path) 

packets_conn = pd.read_csv(conn_in_out_path)

In [4]:
packets_server_name_unique = packets['Server Name'].dropna().unique() 
domain_stream_all = {key: [] for key in packets_server_name_unique} 
for name in domain_stream_all.keys(): # get the dictionary of sni: stream_ids
    stream_ids = packets.loc[packets['Server Name'] == name, 'TCP Stream index'] 
    domain_stream_all[name].extend(stream_ids) 

In [5]:
stream_domain_all = {stream_id: domain for domain, stream_ids in domain_stream_all.items() for stream_id in stream_ids} # reverse the dictionary

In [6]:
packets['Server Name'] = packets['TCP Stream index'].map(stream_domain_all)

In [8]:
packets_statistic['Server Name'] = packets_statistic['Stream ID'].map(stream_domain_all)

In [12]:
packets_conn['Server Name'] = packets_conn['M Stream ID'].map(stream_domain_all)

In [16]:
print(set(packets_conn['Server Name']))

{'rr3---sn-q4flrnld.googlevideo.com', 'ai.immersivetranslate.com', 'basic-static-server.infinitynewtab.com', 'www.gstatic.com', 'rr5---sn-a5meknzk.googlevideo.com', 'lh3.googleusercontent.com', 'api.infinitynewtab.com', 'beacons4.gvt2.com', 'clientservices.googleapis.com', 'accounts.youtube.com', 'beacons3.gvt2.com', 'rr5---sn-a5mekn6z.googlevideo.com', 'jnn-pa.googleapis.com', 'beacons2.gvt2.com', 'infinityicon.infinitynewtab.com', 'content-autofill.googleapis.com', 'clients2.google.com', 'play.google.com', 'android.clients.google.com', 'clients4.google.com', 'safebrowsing.googleapis.com', 'update.googleapis.com', 'www.google.com', 'infinity-permanent.infinitynewtab.com', 'rr3---sn-a5mekndz.googlevideo.com', 'www.youtube.com', 'memex-pa.googleapis.com', 'youtube.com', 'weatheroffer.com', 'rr1---sn-a5mlrnls.googlevideo.com', 'optimizationguide-pa.googleapis.com', 'www.googleapis.com', 'i.ytimg.com', 'beacons.gcp.gvt2.com', 'rr1---sn-q4fl6nsk.googlevideo.com', 'yt3.ggpht.com', 'accounts